# Strassen's Algorithm

In this file, we will explore several sub-cubic matrix multiplication algorithms. The first is Strassen's Algorithm. We will first compare this implementation of Strassen's Algorithm and Naive Implementation.

In [1]:
# Because Strassen's Algorithm is basically "faster" matrix multiplication so 2-by-2 matrices,
# we define a new type of 2-by-2 matrices.
using StaticArrays
struct Matrix1{T}
    a::SArray{Tuple{2,2},T,2,4}
end

In [2]:
# Overwrite/Implement basic arithmetics of type Matrix1
import Base.+
import Base.-
import Base.*
import Base.zero
import Base.show

function +(A::Matrix1, B::Matrix1)
    return Matrix1(A.a+B.a)
end
function -(A::Matrix1, B::Matrix1)
    return Matrix1(A.a-B.a)
end
function *(A::Matrix1, c::Float64)
    return Matrix1(A.a*c)
end
function *(c::Float64, A::Matrix1)
    return Matrix1(c*A.a)
end
function zero(A::Matrix1)
    return Matrix1(SMatrix{2,2}(zero(A.a[1,1]), zero(A.a[1,1]), zero(A.a[1,1]), zero(A.a[1,1])))
end
# To aviod excessive printout of Matrix1{Matrix1{Matrix1{...}}}, we overwrite its "show".
function show(io::IO, A::Matrix1)
    write(io, "number of entries: ", string(div(sizeof(A),sizeof(Float64))))
end

show (generic function with 332 methods)

In [3]:
# Define BruteForce recursively: we need such implementation of recursive calls 
# because matrix is actually of type Matrix1{Matrix1{Matrix1{...}}}
# Ideally, this implementation should still have the expected asymptotic behavior in terms of runtime: O(n^3)
function BruteForce(A::Any, B::Any)
    A*B
end

function BruteForce(A::Matrix1, B::Matrix1)
    C11 = BruteForce(A.a[1,1],B.a[1,1]) + BruteForce(A.a[1,2],B.a[2,1])
    C12 = BruteForce(A.a[1,1],B.a[1,2]) + BruteForce(A.a[1,2],B.a[2,2])
    C21 = BruteForce(A.a[2,1],B.a[1,1]) + BruteForce(A.a[2,2],B.a[2,1])
    C22 = BruteForce(A.a[2,1],B.a[1,2]) + BruteForce(A.a[2,2],B.a[2,2])
    return Matrix1(SMatrix{2,2}(C11,C21,C12,C22))
end

BruteForce (generic function with 2 methods)

In [4]:
# Then we define Strassen.
# Thanks to this new data structure, we no longer need slicing.
function *(A::Matrix1, B::Matrix1) #Strassen
    M1 = (A.a[1,1]+A.a[2,2])*(B.a[1,1]+B.a[2,2])
    M2 = (A.a[2,1]+A.a[2,2])*B.a[1,1]
    M3 = A.a[1,1]*(B.a[1,2]-B.a[2,2])
    M4 = A.a[2,2]*(B.a[2,1]-B.a[1,1])
    M5 = (A.a[1,1]+A.a[1,2])*B.a[2,2]
    M6 = (A.a[2,1]-A.a[1,1])*(B.a[1,1]+B.a[1,2])
    M7 = (A.a[1,2]-A.a[2,2])*(B.a[2,1]+B.a[2,2])
    return Matrix1(SMatrix{2,2}(M1+M4-M5+M7,M2+M4,M3+M5,M1-M2+M3+M6))
end

* (generic function with 365 methods)

In [5]:
# Verify that both multiplications works
A = Matrix1(SMatrix{2,2}(10.0, 20.0, 3, 4))
BruteForce(A, A).a

2×2 SArray{Tuple{2,2},Float64,2,4}:
 160.0  42.0
 280.0  76.0

In [6]:
(A*A).a

2×2 SArray{Tuple{2,2},Float64,2,4}:
 160.0  42.0
 280.0  76.0

In [7]:
# Try multiply matrices with larger sizes.
A = Matrix1(SMatrix{2,2}(10.0, 20.0, 3, 4))
for i = 1:5
    A = Matrix1(SMatrix{2,2}(A, A, A, A))
end
@elapsed B=BruteForce(A, A)

2.097593085

In [8]:
A = Matrix1(SMatrix{2,2}(10.0, 20.0, 3, 4))
for i = 1:5
    A = Matrix1(SMatrix{2,2}(A, A, A, A))
end
@elapsed A*A

4.30747746

In [9]:
# Do not run this cell twice
using NBInclude
@nbinclude("analyze.ipynb")

estimateComplexity (generic function with 2 methods)

Having the algorithms implemented, we try to analyze the number of calculations used by each of them. We expect that the number of multiplication done by BruteForce is $8^4 = 4096$; that by Strassen's Algo is $7^4 = 2401$.

In [10]:
# Define a 16-by-16 matrix
A = Matrix1(SMatrix{2,2}(MyFloat64(10.0), MyFloat64(20.0), MyFloat64(3), MyFloat64(4)))
A = Matrix1(SMatrix{2,2}(A, A, A, A))
A = Matrix1(SMatrix{2,2}(A, A, A, A))
A = Matrix1(SMatrix{2,2}(A, A, A, A))

number of entries: 256

In [11]:
# Then we analyze the number of each type of calculations.
@analyzeMyFloat BruteForce(A,A)

Number of additions: 3840
Number of subtractions: 0
Number of multiplications: 4096
Number of divisions: 0


number of entries: 256

In [12]:
@analyzeMyFloat A*A

Number of additions: 8580
Number of subtractions: 4290
Number of multiplications: 2401
Number of divisions: 0


number of entries: 256

Noticing that the number of multiplication done by BruteForce is (exponentially) more than Strassen's Algorithm, we try to use polynomial ring to amplify this difference.

In [13]:
# using Pkg; Pkg.add("Polynomials")
using Polynomials

In [14]:
A = Matrix1(SMatrix{2,2}(Poly(rand(1000)), Poly(rand(1000)), Poly(rand(1000)), Poly(rand(1000))))
A = Matrix1(SMatrix{2,2}(A, A, A, A))
A = Matrix1(SMatrix{2,2}(A, A, A, A))
A = Matrix1(SMatrix{2,2}(A, A, A, A))

number of entries: 1

In [17]:
@elapsed BruteForce(A, A)

6.406702425

In [18]:
@elapsed A*A

0.322867743

# APPROXIMATION

Theorem (BCS, Proposition 15.10, Informal). If multiplication of two square matrix of size n can be approximated with r multiplications, then it can be computed exactly with r multiplications.

This gives rise to many faster (but probably not yet practical) matrix multiplication algorithms, including the Coppersmith-Winograd Matrix Multiplication Algorithm (which is one of the fastest matrix multiplication algorithm we have right now, and achieves asymptotically $O(n^{2.38}))$. 

Here, we show an interesting identity (Schonhage's identity) that can be modified into an approximation algorithm for matrix multiplication in $O(n^{2.55})$ multiplications. Other faster matrix multiplications usually involves other identities regarding the tensor products. For detail, we recommend interested readers to read relevant papers in the References section.

Specifically, it approximates an outer product of a 4-by-1 vector and an 1-by-4 vector and an inner product of an 1-by-9 vector and a 9-by-1 vector with 17 multiplications (ignore scalar multiplications). A naive implementation for calculating these two products needs $16+9=25$ multiplications)

In [19]:
# Because we are multiplying two different pairs at the same time, we define new data structures for pairs of vectors.
using StaticArrays
struct MatrixPair1{T}
    a::SArray{Tuple{4,1},T,2,4}
    x::SArray{Tuple{1,9},T,2,9}
end
struct MatrixPair2{T}
    b::SArray{Tuple{1,4},T,2,4}
    y::SArray{Tuple{9,1},T,2,9}
end

struct MatrixPair3{T}
    c::SArray{Tuple{4,4},T,2,16}
    z::SArray{Tuple{1,1},T,2,1}
end

In [20]:
# Overwrite basic arithmetics
import Base.+
import Base.-
import Base.*
import Base.zero
import Base.show

function +(A::MatrixPair1, B::MatrixPair1)
    return MatrixPair1(A.a+B.a, A.x+B.x)
end
function -(A::MatrixPair1, B::MatrixPair1)
    return MatrixPair1(A.a-B.a, A.x-B.x)
end
function *(A::MatrixPair1, c::Float64)
    return MatrixPair1(A.a*c, A.x*c)
end
function *(c::Float64, A::MatrixPair1)
    return MatrixPair1(c*A.a, c*A.x)
end
function zero(A::MatrixPair1)
    return MatrixPair1(SMatrix{4,1}(fill(zero(A.a[1,1]),4)), SMatrix{1,9}(fill(zero(A.x[1,1]),9)))
end
function show(io::IO, A::MatrixPair1)
    write(io, "number of entries: ", string(div(sizeof(A),sizeof(Float64))))
end


function +(A::MatrixPair2, B::MatrixPair2)
    return MatrixPair2(A.b+B.b, A.y+B.y)
end
function -(A::MatrixPair2, B::MatrixPair2)
    return MatrixPair2(A.b-B.b, A.y-B.y)
end
function *(A::MatrixPair2, c::Float64)
    return MatrixPair2(A.b*c, A.y*c)
end
function *(c::Float64, A::MatrixPair2)
    return MatrixPair2(c*A.b, c*A.y)
end
function zero(A::MatrixPair2)
    return MatrixPair2(SMatrix{1,4}(fill(zero(A.b[1,1]),4)), SMatrix{9,1}(fill(zero(A.y[1,1]),9)))
end
function show(io::IO, A::MatrixPair2)
    write(io, "number of entries: ", string(div(sizeof(A),sizeof(Float64))))
end

function +(A::MatrixPair3, B::MatrixPair3)
    return MatrixPair3(A.c+B.c, A.z+B.z)
end
function -(A::MatrixPair3, B::MatrixPair3)
    return MatrixPair3(A.c-B.c, A.z-B.z)
end
function *(A::MatrixPair3, c::Float64)
    return MatrixPair3(A.c*c, A.z*c)
end
function *(c::Float64, A::MatrixPair3)
    return MatrixPair3(c*A.c, c*A.z)
end
function zero(A::MatrixPair3)
    return MatrixPair3(SMatrix{4,4}(fill(zero(A.c[1,1]),16)), SMatrix{1,1}(fill(zero(A.z[1,1]),1)))
end
function show(io::IO, A::MatrixPair3)
    write(io, "number of entries: ", string(div(sizeof(A),sizeof(Float64))))
end

show (generic function with 344 methods)

In [21]:
Pair1 = MatrixPair1(SMatrix{4,1}(1,1,1,1), SMatrix{1,9}(fill(1,9)))

number of entries: 13

In [22]:
Pair12 = MatrixPair1(SMatrix{4,1}(fill(Pair1,4)), SMatrix{1,9}(fill(Pair1,9)))

number of entries: 169

In [23]:
Pair2 = MatrixPair2(SMatrix{1,4}(1,1,1,1), SMatrix{9,1}(fill(1,9)))

number of entries: 13

In [24]:
Pair22 = MatrixPair2(SMatrix{1,4}(fill(Pair2,4)), SMatrix{9,1}(fill(Pair2,9)))

number of entries: 169

In [25]:
function construct(P1::MatrixPair1)
    return MatrixPair3(SMatrix{4,4}(fill(construct(P1.a[1,1]),4,4)), SMatrix{1,1}(fill(construct(P1.x[1,1]),1)))
end

function construct(P1)
    zero(P1)
end

construct (generic function with 2 methods)

In [26]:
X = Pair22-Pair22

number of entries: 169

In [27]:
X.b[1,1].b

1×4 SArray{Tuple{1,4},Int64,2,4}:
 0  0  0  0

Here we overwrite $*$ for `(MatrixPair1)*(MatrixPair2)`. $\epsilon$ is the approximation factor. The multiplication result is `X*Y + O(ϵ)`. One should note that `(MatrixPair2)*(MatrixPair3)` and `(MatrixPair3)*(MatrixPair1)` can also be defined, using the Schnohage's identity.

In [28]:
function *(P1::MatrixPair1, P2::MatrixPair2)
    # A, B, C(=AB), X, Y, Z(=XY) are all 2D-arrays.
    ϵ = 0.001
    C = fill(construct(P1.a[1,1]), 4, 4)
    Z = fill(construct(P1.a[1,1]), 1, 1)
    
    # 9 multiplications in the following double for-loop
    for i=1:3
        for j=1:3
            temp = (P1.a[i,1]+ϵ*P1.x[1,3*i+j-3])*(P2.b[1,j]+ϵ*P2.y[3*i+j-3,1])
            C[i,j] += temp
            Z[1,1] += temp*ϵ^(-2)
        end
    end
    # 3 multiplications in the following loop
    for i=1:3
        y = P2.y[3*i,1] + P2.y[3*i-1,1] + P2.y[3*i-2,1]
        temp = P1.a[i,1]*(P2.b[1,4]-ϵ*y)
        C[i,4] += temp
        Z[1,1] += temp*ϵ^(-2)
    end
    
    # 3 multiplications in the following loop
    for j=1:3
        x = P1.x[1,j] + P1.x[1,j+3] + P1.x[1,j+6]
        temp = (P1.a[4,1]-ϵ*x)*P2.b[1,j]
        C[4,j] += temp
        Z[1,1] += temp*ϵ^(-2)
    end
    
    # one multiplication
    temp = P1.a[4,1]*P2.b[1,4]
    C[4,4] += temp
    Z[1,1] += temp*ϵ^(-2)
    
    # one multiplication
    Z[1,1] -= sum(P1.a)*sum(P2.b)*ϵ^(-2)
    
    MatrixPair3(SMatrix{4,4}(C), SMatrix{1,1}(Z))
    
end

* (generic function with 378 methods)

In [29]:
# To test this identity, we try to define a random element of type MatrixPair1{MatrixPair1{...}}.
function build1(f)
    A = fill(zero(f()), 4, 1)
    B = fill(zero(f()), 1, 9)
    for i = 1:4
        A[i, 1] = f()
    end
    for i = 1:9
        B[1, i] = f()
    end
    return MatrixPair1(SMatrix{4,1}(A),SMatrix{1,9}(B))
end

build1 (generic function with 1 method)

In [30]:
build1(rand)

number of entries: 13

In [31]:
build1(()->(build1(()->build1(rand))))

number of entries: 2197

In [32]:
# To test this identity, we try to define a random element of type MatrixPair2{MatrixPair2{...}}.
function build2(f)
    A = fill(zero(f()), 1, 4)
    B = fill(zero(f()), 9, 1)
    for i = 1:4
        A[1, i] = f()
    end
    for i = 1:9
        B[i, 1] = f()
    end
    return MatrixPair2(SMatrix{1, 4}(A),SMatrix{9, 1}(B))
end

build2 (generic function with 1 method)

In [33]:
X = build1(()->build1(rand))

number of entries: 169

In [34]:
Y = build2(()->build2(rand))

number of entries: 169

In [35]:
Z = X*Y

number of entries: 289

In [36]:
construct(X)

number of entries: 289

With random entries, one should see that the difference in every entry is $O(\epsilon)$ where $\epsilon = 10^{-3}$.

In [37]:
Z.c[1,1].c - X.a[1,1].a * Y.b[1,1].b

4×4 SArray{Tuple{4,4},Float64,2,16}:
 0.0008749    0.00194436    0.00225486   -6.60592e-5 
 0.000484977  0.000598796   0.00139713    0.000486833
 0.000810899  0.00105452    0.00208996    0.000569894
 0.000103301  6.71705e-5   -0.000829518   0.00113678 

In [38]:
Z.z - X.x * Y.y

1×1 SArray{Tuple{1,1},MatrixPair3{Float64},2,1}:
 number of entries: 17

# References

*Papers:*

Schönhage, Arnold. "Partial and total matrix multiplication." SIAM Journal on Computing 10.3 (1981): 434-455.

Anderson, Matthew, and Siddharth Barman. "The Coppersmith-Winograd Matrix Multiplication Algorithm." (2009).

Bürgisser, Peter, Michael Clausen, and Mohammad A. Shokrollahi. Algebraic complexity theory. Vol. 315. Springer Science & Business Media, 2013.

Strassen, Volker. "Gaussian elimination is not optimal." Numerische mathematik 13.4 (1969): 354-356.

*Packages:* 

StaticArrays, Polynomials